<a href="https://colab.research.google.com/github/HedersonSantos/Noticias/blob/main/ulmfit/ULMFit_Noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip3 install fastai

In [13]:
from fastai import *
from fastai.text import *
import pandas as pd
from pathlib import Path
from datetime import datetime
from google.colab import drive

In [4]:
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Colab\ Notebooks


Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks


In [9]:
#%cd amostra_6/
#!unzip amostra6.zip
#!ls

amostra6.csv  amostra6.zip


In [7]:
source_folder = './amostra_news_integrada/amostra_' 
destination_folder = './ulmfit_resp' 
_id=6

In [15]:
#df = pd.read_csv(source_folder+str(6)+'/amostra6.csv')


In [34]:
bs = 128
lang = 'pt'
data = (TextList.from_csv(source_folder+str(_id),'amostra6.csv', cols='text')
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [38]:
data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

(60000, 71144)

In [8]:
bs = 128
lang = 'pt'
data = load_data(source_folder+str(_id), f'{lang}_databunch', bs=bs)

In [9]:
data.show_batch()

idx,text
0,"a letra "" z "" , uma referência ao ex - jogador . xxmaj tradicionalmente , a faixa de capitão do xxmaj santos tem a letra "" c "" ( de capitão ) . xxmaj contra o xxmaj corinthians , foi a primeira vez que o time se apresentou com um modelo diferente . o capitão do jogo , xxmaj ricardo xxmaj oliveira , fez o gol da vitória santista"
1,"por que não se apura toda a merenda ? "" , disse xxmaj capez em coletiva após a xxup cpi , repetindo o que já havia dito durante a sessão . a merenda seca fornecida pelo governo foi o que motivou a abertura de uma apuração do xxmaj tribunal de xxmaj contas do xxmaj estado que apontou uma série de problemas nas refeições das escolas estaduais . a fala do"
2,"xxmaj brasileiro na 2ª rodada , o xxmaj corinthians só voltou à ponta na semana passada . xxmaj neste intervalo , viu xxmaj atlético - xxup mg ( sete rodadas ) , xxmaj sport ( cinco rodadas ) , xxmaj atlético - xxup pr ( três rodadas ) e xxmaj são xxmaj paulo ( uma rodada ) na primeira colocação . o histórico do torneio indica que a"
3,"receita do muffin . xxmaj na cadeia , presa injustamente no lugar da irmã , a verdadeira xxmaj ernestina diz que xxmaj matilde sempre roubava tudo que era dela , até mesmo os namorados . xxmaj ela se lembra de quando era jovem e xxmaj matilde saiu com seu primeiro amor no lugar dela . xxmaj ernestina diz que esse tempo injusto na cadeia fez ela perceber que não pode"
4,"por abusar sexualmente de dois menores e pela posse de pornografia infantil . xxmaj ele cumpre uma pena de reclusão de cinco anos . xxmaj apesar disso , xxmaj xxunk só foi desligado formalmente da igreja em março deste ano . o ex - padre também deve ir a julgamento após acusações de um terceiro abuso sexual . xxmaj todos os casos teriam ocorrido entre 2008 e 2010 . xxmaj"


In [10]:
#o modelo foi treinado para o inglês. Como os dados estão em português, utilizaremos a flag pretrained = False
#e rodaremos um ciclo de treinamento do modelo para o português.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if device == 'cpu':
  learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False)#.to_fp16()
else:
  learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [14]:
lr = 1e-2
inicio = datetime.now()
learn.unfreeze() #como iremos treinar o modelo para definirmos novos parâmetros, liberarmos os parâmetros de treinamento para redefinição (descongelamento dos parâmetros de treinamento)
learn.fit_one_cycle(12, lr, moms=(0.8,0.7)) 
print('duracao: ', datetime.now()-inicio)

epoch,train_loss,valid_loss,accuracy,time


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1702: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  if p.grad is not None:


RuntimeError: ignored

In [ ]:
#Salvando o modelo pre-treinado e o vocabulário (vocab) 
lm_fns = [f'{lang}_model', f'{lang}_vocab']
pathlms = Path(destination_folder)
learn.to_fp32().save(pathlms/lm_fns[0], with_opt=False)
learn.data.vocab.save(pathlms/(lm_fns[1] + '.pkl'))